In [ ]:
import os
import asyncio

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

: 

In [ ]:
# #register an account at ngrok.com and create an authtoken and place it here
# await asyncio.gather(
#     run_process(['ngrok', 'config', 'add-authtoken','2clxYizDjD85qMJyCvTK06FPit2_7HW3FyD53n6M6mHcCAapk'])
# )

# await asyncio.gather(
#     run_process(['ollama', 'serve']),
#     run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
#     # run_process(['ngrok', 'http', '--log', 'stderr', '--domain', 'renewed-shark-pro.ngrok-free.app', '11434']),
# )